In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [0]:
#importando o dataset
iris = spark.read.csv("/FileStore/tables/iris.csv",inferSchema=True, header=True)
irisTreino, irisTeste = iris.randomSplit([0.7,0.3])

In [0]:
#definindo e criando um vector passeado nas colunas
vector = VectorAssembler(inputCols=["sepallength", "sepalwidth", "petallength", "petalwidth"], outputCol="independente")

In [0]:
#
indexer = StringIndexer(inputCol="class", outputCol="label")

In [0]:
#
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[4, 5, 4, 3], featuresCol="independente", labelCol="label")

In [0]:
#
pipeline = Pipeline(stages=[vector, indexer, mlp]) 

In [0]:

grid = ParamGridBuilder().addGrid(mlp.maxIter, [10, 100, 1000]).addGrid(mlp.layers, [[4, 5, 4, 3], [4, 4, 3]]).build()
crossval = CrossValidator(estimator=pipeline,estimatorParamMaps=grid,evaluator=MulticlassClassificationEvaluator(),numFolds=3)  

In [0]:
modelo = crossval.fit(irisTreino)

In [0]:
previsao = modelo.transform(irisTeste)
previsao.show()

+-----------+----------+-----------+----------+---------------+-----------------+-----+--------------------+--------------------+----------+
|sepallength|sepalwidth|petallength|petalwidth|          class|     independente|label|       rawPrediction|         probability|prediction|
+-----------+----------+-----------+----------+---------------+-----------------+-----+--------------------+--------------------+----------+
|        4.6|       3.6|        1.0|       0.2|    Iris-setosa|[4.6,3.6,1.0,0.2]|  0.0|[266.574390495330...|[1.0,6.5761242368...|       0.0|
|        4.8|       3.0|        1.4|       0.1|    Iris-setosa|[4.8,3.0,1.4,0.1]|  0.0|[280.944553530138...|[1.0,9.1595673944...|       0.0|
|        4.9|       2.5|        4.5|       1.7| Iris-virginica|[4.9,2.5,4.5,1.7]|  1.0|[-470.30543436446...|[1.03461849349861...|       2.0|
|        4.9|       3.0|        1.4|       0.2|    Iris-setosa|[4.9,3.0,1.4,0.2]|  0.0|[266.574390495330...|[1.0,6.5761242368...|       0.0|
|        4.9|

In [0]:

performance = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(previsao)
print(acuracia)

0.9142857142857143
